In [ ]:
pip install mediapipe opencv-python


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# import os
# import zipfile


# # tải file zip từ github
# zip_path = "/content/drive/MyDrive/DATASET_VSL/compressed_jpg.zip"

# extract_path = "/content/sign_language_digits"

# # giản nén file zip
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#   zip_ref.extractall(extract_path)
# print( "Đã giải nén thànhh công tại:", extract_path)

In [ ]:
from collections import defaultdict
import os
# đường dẫn tới thư mục ảnh
root_dir = "/content/drive/MyDrive/DATASET_VSL/compressed_jpg"
# Biến đếm tổng số ảnh
total_images = 0
# Đếm số ảnh trong mỗi lớp
count_dict = defaultdict(int)

# Duyệt qua các thư mục con trong thư mục gốc
for label in sorted(os.listdir(root_dir)):
  label_dir = os.path.join(root_dir, label)
  if os.path.isdir(label_dir):
    count = len([f for f in os.listdir(label_dir) if f.endswith(".JPG")])
    count_dict[label] = count
    total_images += count

# in ra tổng
print("Tổng số ảnh trong toàn bộ dataset:", total_images)
# in ra ket qua
for label, count in count_dict.items():
  print(f"Lớp {label}: {count} ảnh")

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
import shutil
# ==== CẤU HÌNH ====
root_dir = '/content/drive/MyDrive/DATASET_VSL/compressed_jpg'  # thư mục chứa ảnh gốc
output_dir = '/content/augmented_output'  # nơi lưu ảnh sau tăng cường
target_count = 400                  # số lượng mong muốn mỗi lớp
image_size = (128, 128)             # kích thước chuẩn hóa ảnh

# ==== TẠO THƯ MỤC ĐẦU RA NẾU CHƯA CÓ ====
os.makedirs(output_dir, exist_ok=True)

# ==== CẤU HÌNH BỘ TĂNG CƯỜNG ẢNH ====
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=5,
    zoom_range=0.1,
    brightness_range=(0.8, 1.2),      # đa dạng ánh sáng
    horizontal_flip=False,            # không lật ảnh trái–phải
    fill_mode='nearest'
)

# ==== XỬ LÝ TỪNG LỚP ====
for folder in sorted(os.listdir(root_dir)):
    folder_path = os.path.join(root_dir, folder)
    output_class_dir = os.path.join(output_dir, folder)
    os.makedirs(output_class_dir, exist_ok=True)

    if os.path.isdir(folder_path):
        image_files = [
            f for f in os.listdir(folder_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png')) and not f.startswith('aug_')
        ]
        current_count = len(image_files)
        print(f"\n {folder}: {current_count} ảnh gốc")

        # === Bước 1: Copy ảnh gốc đã resize ===
        for img_name in image_files:
            src_path = os.path.join(folder_path, img_name)
            dst_path = os.path.join(output_class_dir, img_name)

            try:
                img = Image.open(src_path).convert('RGB')
                img = img.resize(image_size)
                img.save(dst_path)
            except Exception as e:
                print(f" Lỗi khi copy ảnh {src_path}: {e}")

        # === Bước 2: Tăng cường nếu chưa đủ ===
        if current_count >= target_count:
            print(" Đã đủ ảnh, không cần tăng cường.")
            continue

        print(f" Đang tăng cường thêm {target_count - current_count} ảnh...")

        i = 0
        while current_count + i < target_count:
            img_name = image_files[i % current_count]
            img_path = os.path.join(folder_path, img_name)

            try:
                img = Image.open(img_path).convert('RGB')
                img = img.resize(image_size)
                x = img_to_array(img)
                x = np.expand_dims(x, axis=0)

                for batch in datagen.flow(x, batch_size=1):
                    aug_img = array_to_img(batch[0])
                    save_name = f"aug_{i}_{img_name}"
                    save_path = os.path.join(output_class_dir, save_name)
                    aug_img.save(save_path)
                    i += 1
                    if current_count + i >= target_count:
                        break

            except Exception as e:
                print(f" Lỗi khi tăng cường ảnh {img_path}: {e}")

print("\n Hoàn tất tăng cường dữ liệu. Tất cả ảnh đã lưu tại:", output_dir)


In [ ]:
import cv2
import mediapipe as mp
import csv
from tqdm import tqdm
from collections import defaultdict
from google.colab.patches import cv2_imshow

# === Khởi tạo model ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)

# === Đường dẫn thư mục và output ===
data_dir = "/content/augmented_output"
output_csv = "/content/hand_keypoints_dataset.csv"

# === Biến đếm ===
valid_count = 0
invalid_count = 0
invalid_per_class = defaultdict(int)

# === Ghi ra file CSV ===
with open(output_csv, mode='w', newline='') as f:
    writer = csv.writer(f)

    # Header CSV
    header = ['label']
    for i in range(21):
        header.extend([f'x{i}', f'y{i}', f'z{i}'])
    writer.writerow(header)

    # Duyệt từng lớp
    for label in sorted(os.listdir(data_dir)):
        label_path = os.path.join(data_dir, label)
        if not os.path.isdir(label_path):
            continue

        print(f" Đang xử lý lớp {label}...")

        image_list = [
            f for f in os.listdir(label_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]

        for img_name in tqdm(image_list):
            img_path = os.path.join(label_path, img_name)
            image = cv2.imread(img_path)
            if image is None:
                invalid_per_class[label] += 1
                continue

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)

            if results.multi_hand_landmarks:
                hand_landmarks = results.multi_hand_landmarks[0]
                keypoints = []
                for lm in hand_landmarks.landmark:
                    keypoints.extend([lm.x, lm.y, lm.z])
                row = [label] + keypoints
                writer.writerow(row)
                valid_count += 1
            else:
                invalid_count += 1
                invalid_per_class[label] += 1

# === In kết quả ===
print(f"\nTrích xuất xong! Ảnh hợp lệ: {valid_count:,}, Bị bỏ qua: {invalid_count:,}")
print(" Số ảnh bị bỏ qua theo từng lớp:")
for label in sorted(invalid_per_class.keys()):
    print(f" - {label}: {invalid_per_class[label]} ảnh")
